In [103]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle

In [2]:
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names = []
for f in tqdm(shp_files):
    x = geopandas.read_file(f, engine="pyogrio")
    places_names.append(x.name)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:38<00:00,  1.66s/it]


In [6]:
names = pd.concat(places_names, axis=0)

In [30]:
names.head(2)

0    元记电气焊补胎钣金
1       元汇津通卸载
Name: name, dtype: object

In [11]:
names.to_csv('tianjin.csv', index=False)

In [11]:
sf = shapefile.Reader('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/天津市_公共设施_202209.shp')

In [ ]:
for s in sf:
    print(s.record[0])

In [14]:
import xlrd3

In [15]:
data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/first_2w.xls')

In [16]:
table = data.sheets()[0]          #通过索引顺序获取

In [78]:
texts = []
for i in range(1, table.nrows):
    texts.append(table.row(i)[5].value)
    
texts = pd.DataFrame(texts, columns=['text'])
texts.insert(1, 'places', pd.Series([[]]*len(texts)))

In [79]:
# i.append('t') for i in texts.head(2).places.to_list()

In [85]:
places = []
sparse_matirx = []

for i in tqdm(names.unique()[297604:]):
    
    try:
        result = texts.text.str.contains(i)
        if result.to_numpy().sum() > 0:
            places.append(i)
            sparse_matirx.append(result.to_numpy())
    except Exception as e:
        continue
        # break
        # texts[texts.text.str.contains(i)].places = texts[texts.text.str.contains(i)].places.to_list().append(i)

  0%|                                                                                                                                                                                 | 0/402578 [00:00<?, ?it/s]/tmp/ipykernel_439577/2576573347.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  result = texts.text.str.contains(i)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 402578/402578 [09:05<00:00, 738.59it/s]


In [105]:
print(len(places), len(sparse_matirx))
with open('place2index.pickle', 'wb') as f:
    pickle.dump([places, sparse_matirx], f)

816 816


In [102]:
name2num = []
for i,p in enumerate(places):
    name2num.append(sparse_matirx[i].sum())

name2num = np.array(name2num)
order_num = np.flip(np.argsort(name2num))
np_places = np.array(places)
for p, n in zip(np_places[order_num][:100], name2num[order_num][:100]):
    print(p, ":", n)

天津 : 6174
天津之眼 : 539
天津市 : 190
滨海 : 173
美食 : 167
海河 : 160
天津大学 : 140
天津博物馆 : 121
天津站 : 112
A+ : 93
天津失恋博物馆 : 88
图书馆 : 78
文化街 : 75
古文化街 : 74
饭 : 72
天津街 : 61
摩天轮 : 57
和平 : 54
瓷房子 : 53
滨海新区 : 53
滨海国际 : 53
过大年 : 50
酒店 : 49
天津古文化街 : 47
天津之眼摩天轮 : 42
江道 : 40
滨江道 : 39
天津西站 : 37
解放桥 : 35
天津自然博物馆 : 34
网红 : 30
佛 : 29
大胡同 : 29
花园 : 29
饼果子 : 27
河东 : 26
体育馆 : 24
和平区 : 24
天津记忆 : 23
南市 : 23
星巴克 : 22
食堂 : 21
河西 : 21
大街 : 21
天津工业大学 : 19
天津外国语大学 : 18
天津美术馆 : 18
牛肉 : 18
鼓楼 : 17
天津科技大学 : 17
幸福 : 17
南开区 : 16
解放北路 : 16
梅江 : 15
恒隆广场 : 15
天津水上公园 : 15
天津动物园 : 15
河西区 : 14
和平路 : 14
时光 : 14
公寓 : 14
天津师范大学 : 14
河北工业大学 : 13
白楼 : 12
西青 : 12
小白楼 : 12
医院 : 12
望海楼 : 12
河东区 : 12
饺 : 11
阳光 : 11
马场 : 11
大理道 : 11
休闲 : 11
杨柳青古镇 : 11
北洋园 : 11
冬天 : 11
夜色 : 11
宅 : 11
天津港 : 10
水西公园 : 10
时节 : 10
天津商业大学 : 10
天津大剧院 : 10
西沽 : 10
西青区 : 10
天塔 : 10
天津滨海国际机场 : 9
奶茶 : 9
宁园 : 9
金融街 : 9
津河 : 9
鸡蛋 : 9
渤海 : 9
天津市第四中学 : 9
蓟州区 : 9
国家海洋博物馆 : 9
西沽公园 : 9
俱乐部 : 8
大王 : 8


In [98]:
name2num

array([['烤红薯', '1'],
       ['老豆腐', '1'],
       ['美食', '167'],
       ...,
       ['东疆湾景区', '1'],
       ['河东公园', '4'],
       ['天津盆景园', '1']], dtype='<U21')

In [47]:
for i in texts.places.values:
    if len(i) > 0:
        print(i)

In [66]:
np.unique(names.values)

NameError: name 'np' is not defined

In [39]:
texts.text.str.contains('天津').to_numpy().sum()

6174

In [65]:
texts

,text,places,维修,公司,工厂,OPPO
0,River lounge opening party @ The St. Regis Tia...,[],False,False,False,False
1,> 苏帮袁Sue Cuisine <\n#汤圆好食##TangYuan's Life#\n\...,[],False,False,False,False
2,<年轻人>构想前程，<老年人>喜爱回忆。\n《1956年》<憨厚朴实听话爱干活>，农村刚出来...,[],False,False,False,False
3,<锦胜雾森>天津津南区蓝光水果侠喷雾造景项目，景区乐园雾森造雾，人造仙境设备制造公司,[],False,True,False,False
4,天津,[],False,False,False,False
...,...,...,...,...,...,...
6237,#春游季##天津动物园#,[],False,False,False,False
6238,#春游季##五一放假4天##3月去哪儿玩# 今天气温降低，但是天光大好。春季如何不赏花？——...,[],False,False,False,False
6239,#春运正式开启# 天津机场,[],False,False,False,False
6240,#春韵##宝宝的少女心# 感谢小天\n@天津大学 大美校园活动的奖品\n卡贴好可爱 好多天大...,[],False,False,False,False
